In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb

from os import listdir
from tqdm import tqdm

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, scale 
from sklearn.model_selection import StratifiedKFold, ParameterSampler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

input_path = "../input/"
output_path = "../output/"

In [2]:
npys = ([f for f in sorted(listdir(output_path)) if f.endswith("_oof.npy") and f[0]!="!" and "nn" in f 
        and (f.split("_")[2]=='oof.npy' or float(f.split("_")[2]) > 0.921)])
npys

['nn_0.9163_oof.npy',
 'nn_0_0.92162_oof.npy',
 'nn_0_0.92191_oof.npy',
 'nn_0_0.92225_oof.npy',
 'nn_0_0.92226_oof.npy',
 'nn_0_0.92269_oof.npy',
 'nn_0_0.92273_oof.npy',
 'nn_0_0.9229_oof.npy',
 'nn_0_0.92345_oof.npy',
 'nn_1_0.92213_oof.npy',
 'nn_1_0.92217_oof.npy',
 'nn_1_0.92266_oof.npy',
 'nn_1_0.92273_oof.npy',
 'nn_1_0.92294_oof.npy',
 'nn_1_0.92311_oof.npy',
 'nn_1_0.923_oof.npy',
 'nn_2_0.92128_oof.npy',
 'nn_2_0.9228_oof.npy',
 'nn_2_0.92314_oof.npy',
 'nn_2_0.92316_oof.npy',
 'nn_3_0.92114_oof.npy',
 'nn_3_0.92172_oof.npy',
 'nn_3_0.92235_oof.npy',
 'nn_3_0.92269_oof.npy',
 'nn_3_0.92287_oof.npy',
 'nn_3_0.92333_oof.npy',
 'nn_4_0.92263_oof.npy',
 'nn_4_0.9227_oof.npy',
 'nn_4_0.92292_oof.npy',
 'nn_5_0.92241_oof.npy',
 'nn_5_0.92248_oof.npy',
 'nn_5_0.92258_oof.npy',
 'nn_5_0.92286_oof.npy',
 'nn_5_0.92289_oof.npy',
 'nn_aug_0.92303_12_oof.npy']

In [3]:
path = "../kaggle-santander-ctp-2019/predictions/train/"
csvs = [f for f in sorted(listdir(path)) if f.endswith(".csv") and "stacking" not in f][-17:]
csvs

['9197_lightgbm_unique_10x_upsample_0104035943.csv',
 '9199_xgboost_unique_10x_upsample_0104062006.csv',
 '9202_separate_feature_models_threshold_0.84_10fold_04042353.csv',
 '9208_separate_feature_models_threshold_0.80_10fold_07041509.csv',
 '9210_lightgbm_multiplied_counter_10_folds_0604220718.csv',
 '9211_catboost_unbalanced_weights_0304111611.csv',
 '9211_catboost_unique_with_10x_upsample_3103172847.csv',
 '9211_xgboost_multiplied_counter_10_folds_0604015413.csv',
 '9212_catboost_unique_10x_upsample_5_balancer_0204172825.csv',
 '9215_catboost_unique_10x_upsample_3_balancer_0204145755.csv',
 '9218_catboost_excluded_features_10_folds_0804184027.csv',
 '9218_catboost_multiplied_counter_0304011056.csv',
 '9219_catboost_best_parameters_10_folds_0704212230.csv',
 '9222_catboost_multiplied_counter_10_folds_0504034846.csv',
 '9222_catboost_poisson_bootstrap_10_folds_0804042543.csv',
 '9223_catboost_unique_10_fold_20x_upsample_0104001611.csv',
 '9225_catboost_isotonic_09040646.csv']

In [7]:
scaler = StandardScaler() #MinMaxScaler StandardScaler

dt = pd.DataFrame()

df = pd.read_csv(input_path+'train.csv.zip')
df = df[["target"]]

test_filtered = pd.read_pickle(input_path + 'test_filtered.pkl')

cols = []
for file in tqdm(npys):
    tmp = pd.DataFrame()
    
    test = np.load(output_path+file.replace("_oof.npy","_test.npy"))
    test = list(test[test_filtered.index].reshape(-1, 1))
    train = list(np.load(output_path+file).reshape(-1, 1))
    
    col = file[:-4]
    cols.append(col)
    
    tmp[col] = train + test
    tmp[col] = tmp[col].rank()/300000
    
    df[col] = tmp[col][:200000]
    dt[col] = tmp[col][200000:]
    
for file in tqdm(csvs):
    tmp = pd.DataFrame()
    
    test = pd.read_csv((path+file).replace("/train/","/test/"))
    test = list(test.loc[test_filtered.index]["target"].values.reshape(-1, 1))
    train = list(pd.read_csv(path+file)["target"].values.reshape(-1, 1))
    
    col = file[:-4]
    cols.append(col)
    
    tmp[col] = train + test
    tmp[col] = tmp[col].rank()/300000
    
    df[col] = tmp[col][:200000]
    dt[col] = tmp[col][200000:]
    
df["avg"] = df[cols].mean(axis=1)
dt["avg"] = dt[cols].mean(axis=1)

dt.tail()


100%|██████████| 17/17 [02:00<00:00,  7.05s/it]

,nn_0.9163_oof,nn_0_0.92162_oof,nn_0_0.92191_oof,nn_0_0.92225_oof,nn_0_0.92226_oof,nn_0_0.92269_oof,nn_0_0.92273_oof,nn_0_0.9229_oof,nn_0_0.92345_oof,nn_1_0.92213_oof,...,9212_catboost_unique_10x_upsample_5_balancer_0204172825,9215_catboost_unique_10x_upsample_3_balancer_0204145755,9218_catboost_excluded_features_10_folds_0804184027,9218_catboost_multiplied_counter_0304011056,9219_catboost_best_parameters_10_folds_0704212230,9222_catboost_multiplied_counter_10_folds_0504034846,9222_catboost_poisson_bootstrap_10_folds_0804042543,9223_catboost_unique_10_fold_20x_upsample_0104001611,9225_catboost_isotonic_09040646,avg
299995,0.959083,0.914790,0.923507,0.919873,0.920727,0.907053,0.913673,0.920697,0.917063,0.916647,...,0.927697,0.936640,0.937353,0.933417,0.923607,0.931150,0.931690,0.928030,0.931235,0.922583
299996,0.150727,0.157060,0.180470,0.183747,0.204760,0.187807,0.156493,0.176160,0.189790,0.192513,...,0.176777,0.165847,0.188540,0.201797,0.181143,0.188603,0.187920,0.159603,0.174337,0.181315
299997,0.625860,0.717327,0.720957,0.724220,0.680973,0.684923,0.714640,0.694233,0.711743,0.686070,...,0.677520,0.706740,0.762893,0.758610,0.780077,0.763180,0.763963,0.729653,0.764000,0.714572
299998,0.436787,0.418870,0.401343,0.387860,0.419697,0.448987,0.385433,0.423567,0.398750,0.432823,...,0.257977,0.297983,0.400060,0.377217,0.384650,0.381853,0.382180,0.358577,0.370177,0.384958
299999,0.255770,0.596113,0.569017,0.573160,0.577177,0.553157,0.590423,0.566280,0.584293,0.551873,...,0.646813,0.650643,0.580473,0.552273,0.580533,0.577297,0.582080,0.683493,0.578650,0.580427


## AVG

In [8]:
roc_auc_score(df.target, df.avg)
# 0.9252533319134989

0.9252359152263833

In [10]:
dt.tail()

,nn_0.9163_oof,nn_0_0.92162_oof,nn_0_0.92191_oof,nn_0_0.92225_oof,nn_0_0.92226_oof,nn_0_0.92269_oof,nn_0_0.92273_oof,nn_0_0.9229_oof,nn_0_0.92345_oof,nn_1_0.92213_oof,...,9215_catboost_unique_10x_upsample_3_balancer_0204145755,9218_catboost_excluded_features_10_folds_0804184027,9218_catboost_multiplied_counter_0304011056,9219_catboost_best_parameters_10_folds_0704212230,9222_catboost_multiplied_counter_10_folds_0504034846,9222_catboost_poisson_bootstrap_10_folds_0804042543,9223_catboost_unique_10_fold_20x_upsample_0104001611,9225_catboost_isotonic_09040646,avg,clf
299995,0.959083,0.914790,0.923507,0.919873,0.920727,0.907053,0.913673,0.920697,0.917063,0.916647,...,0.936640,0.937353,0.933417,0.923607,0.931150,0.931690,0.928030,0.931235,0.922583,0
299996,0.150727,0.157060,0.180470,0.183747,0.204760,0.187807,0.156493,0.176160,0.189790,0.192513,...,0.165847,0.188540,0.201797,0.181143,0.188603,0.187920,0.159603,0.174337,0.181315,0
299997,0.625860,0.717327,0.720957,0.724220,0.680973,0.684923,0.714640,0.694233,0.711743,0.686070,...,0.706740,0.762893,0.758610,0.780077,0.763180,0.763963,0.729653,0.764000,0.714572,0
299998,0.436787,0.418870,0.401343,0.387860,0.419697,0.448987,0.385433,0.423567,0.398750,0.432823,...,0.297983,0.400060,0.377217,0.384650,0.381853,0.382180,0.358577,0.370177,0.384958,0
299999,0.255770,0.596113,0.569017,0.573160,0.577177,0.553157,0.590423,0.566280,0.584293,0.551873,...,0.650643,0.580473,0.552273,0.580533,0.577297,0.582080,0.683493,0.578650,0.580427,0


## LogReg

In [11]:
%%time

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

df["clf"] = 0
dt["clf"] = 0

i = -1
for train_index, valid_index in skf.split(df[cols], df.target):
    i+=1
    
    X_train = df.loc[train_index, cols]
    X_valid = df.loc[valid_index, cols]

    y_train = df.loc[train_index, "target"]
    y_valid = df.loc[valid_index, "target"]
    
    clf = LogisticRegression(C=0.2,
                             solver="newton-cg", 
                             penalty="l2", 
                             n_jobs=-1, 
                             max_iter=100).fit(X_train, y_train) 
    
    y_pred = clf.predict_proba(X_valid)[:,1] 
    df.loc[valid_index, "clf"] = y_pred
    print(i, "ROC AUC:", round(roc_auc_score(y_valid, y_pred), 5))
    
    dt["clf"] += clf.predict_proba(dt[cols])[:,1] / skf.n_splits
    
print("\nROC AUC:", round(roc_auc_score(df.target, df["clf"]), 5))

# 0 ROC AUC: 0.92511
# 1 ROC AUC: 0.92445
# 2 ROC AUC: 0.92448
# 3 ROC AUC: 0.92458
# 4 ROC AUC: 0.93283
# 5 ROC AUC: 0.92864
# 6 ROC AUC: 0.93127
# 7 ROC AUC: 0.92235
# 8 ROC AUC: 0.92514
# 9 ROC AUC: 0.92289

# ROC AUC: 0.92597
# CPU times: user 10.1 s, sys: 33 s, total: 43.1 s
# Wall time: 2min 39s

# 0 ROC AUC: 0.92516
# 1 ROC AUC: 0.9245
# 2 ROC AUC: 0.92443
# 3 ROC AUC: 0.92452
# 4 ROC AUC: 0.93274
# 5 ROC AUC: 0.92859
# 6 ROC AUC: 0.9312
# 7 ROC AUC: 0.9224
# 8 ROC AUC: 0.92507
# 9 ROC AUC: 0.92297

# ROC AUC: 0.92598
# CPU times: user 10.3 s, sys: 33 s, total: 43.3 s
# Wall time: 2min 36s

0 ROC AUC: 0.92514
1 ROC AUC: 0.9245
2 ROC AUC: 0.92438
3 ROC AUC: 0.92451
4 ROC AUC: 0.93281
5 ROC AUC: 0.92857
6 ROC AUC: 0.93115
7 ROC AUC: 0.92243
8 ROC AUC: 0.92509
9 ROC AUC: 0.92296

ROC AUC: 0.92598
CPU times: user 9.05 s, sys: 32 s, total: 41.1 s
Wall time: 3min 12s


In [12]:
sub = pd.read_csv(input_path+'sample_submission.csv.zip')
sub.loc[test_filtered.index,"target"] = list(dt["clf"])
sub.head(20)

,ID_code,target
0,test_0,0.000000
1,test_1,0.000000
2,test_2,0.000000
3,test_3,0.167291
4,test_4,0.000000
5,test_5,0.000000
6,test_6,0.000000
7,test_7,0.069992
8,test_8,0.000000
9,test_9,0.000000


In [13]:
sub.to_csv(output_path + "best_blend_no_fake.csv", index=False)

In [175]:
!ls ../output/

 best_auc_nn_0.pkl	  nn_0_0.92226_test.npy   nn_3_0.92114_test.npy
 best_auc_nn_1.pkl	  nn_0_0.92269_oof.npy	  nn_3_0.92172_oof.npy
 best_auc_nn_2.pkl	  nn_0_0.92269_test.npy   nn_3_0.92172_test.npy
 best_auc_nn_3.pkl	  nn_0_0.92273_oof.npy	  nn_3_0.92235_oof.npy
 best_auc_nn.pkl	  nn_0_0.92273_test.npy   nn_3_0.92235_test.npy
 best_blend_10.csv	  nn_0_0.9229_oof.npy	  nn_3_0.92269_oof.npy
 best_blend_13.csv	  nn_0_0.9229_test.npy	  nn_3_0.92269_test.npy
 best_blend_19.csv	  nn_0_0.92345_oof.npy	  nn_3_0.92287_oof.npy
 best_blend_25.csv	  nn_0_0.92345_test.npy   nn_3_0.92287_test.npy
 best_blend_33.csv	  nn_0.9163_oof.npy	  nn_3_0.92333_oof.npy
 best_blend_47.csv	  nn_0.9163_test.npy	  nn_3_0.92333_test.npy
 best_blend_7.csv	  nn_1_0.92213_oof.npy	  nn_4_0.92096_oof.npy
 best_blend_8.csv	  nn_1_0.92213_test.npy   nn_4_0.92096_test.npy
 best_blend_.csv	  nn_1_0.92217_oof.npy	  nn_4_0.92263_oof.npy
 best_blend.csv		  nn_1_0.92217_test.npy   nn_4_0.92263_test.npy
'!cat_0.91558_oof.npy'	  

In [176]:
# !kaggle competitions submit -c santander-customer-transaction-prediction -f ../output/best_blend_no_fake.csv -m "0.92598 10-folds reg"

100%|██████████████████████████████████████| 6.17M/6.17M [00:05<00:00, 1.22MB/s]
Successfully submitted to Santander Customer Transaction Prediction